In [17]:
import numpy as np
from astropy.io import fits
import astropy.units as u
from astropy.coordinates import SkyCoord
from photutils.aperture import SkyCircularAperture, aperture_photometry, SkyCircularAnnulus, ApertureStats
from astropy.wcs import WCS
from astropy.stats import SigmaClip

# Load FITS files
filter_paths = [r"C:\Users\Jonathan Chavez\OneDrive\Desktop\GBO-Calibration\calibratedg'.fits", 
                r"C:\Users\Jonathan Chavez\OneDrive\Desktop\GBO-Calibration\calibratedi'.fits"]

#
secondaryStandards = np.array([[209.397542, -9.752132, 11.223, 0.02, 10.771, 0.076], [209.318113, -9.627305, 16.077, 0.029, 14.936, 0.056], [209.360342, -9.411171, 15.534, 0.072, 14.891, 0.064], [209.344599, -9.354614, 16.072, 0.057, 15.411, 0.068], [209.137084, -9.480872, 14.154, 0.051, 13.547, 0.034], [209.21396, -9.422863, 13.238, 0.062, 12.625, 0.034], [209.246385, -9.4099, 15.611, 0.06, 15.045, 0.067], [209.318917, -9.456286, 13.69, 0.048, 11.862, 0.047], [209.442814, -9.517508, 12.974, 0.053, 12.78, 0.052], [209.482814, -9.41469, 14.944, 0.062, 14.213, 0.068], [209.499338, -9.693505, 12.341, 0.025, 11.313, 0.052], [209.512912, -9.590211, 12.507, 0.046, 11.963, 0.041], [209.173385, -9.611714, 15.806, 0.072, 15.111, 0.06], [209.278555, -9.735194, 14.777, 0.014, 14.074, 0.037]])


star_positions = SkyCoord(ra=secondaryStandards[0:14, 0], dec=secondaryStandards[0:14, 1], unit='deg')



# Define aperture radius
r = 0.004*u.deg


for filter_path in filter_paths:

    hdulist = fits.open(filter_path)
    data = hdulist[0].data
    wcs = WCS(hdulist[0].header, key =' ')
    print(wcs)
    #Create errorArray
    errorArray = np.sqrt(np.abs(data))

    # Create CircularAperture objects
    apertures = SkyCircularAperture(star_positions, r)
    annuli = SkyCircularAnnulus(star_positions, 16*u.arcsec, 24*u.arcsec)
    pxlAnnuli = annuli.to_pixel(wcs)


    clipping = SigmaClip(sigma=3.0, maxiters=5, cenfunc='median', stdfunc='std', grow=False)

    #background aperturs and calculations
    bkgData = aperture_photometry(data, pxlAnnuli, method = 'center')
    bkg = ApertureStats(data, pxlAnnuli, sigma_clip = clipping, sum_method = 'center')
    perPxlBkg = bkg.sum/bkg.sum_aper_area


    #the laziest way to find aperture area
    pxlCircles = apertures.to_pixel(wcs)
    aperArea = ApertureStats(data, pxlCircles, sigma_clip = None, sum_method = 'center')


    # Perform aperture photometry
    phot_table = aperture_photometry(data, apertures, error = errorArray, wcs=wcs, method = 'center')

    #update phot_table with background subtracted values
    phot_table['aperture_sum'] = phot_table['aperture_sum'] - perPxlBkg*aperArea.sum_aper_area

    print("results:\n")
    print(phot_table)


    hdulist.close()

WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN'  'DEC--TAN'  
CRVAL : 209.320126444  -9.54338232894  
CRPIX : 2048.0  2048.0  
CD1_1 CD1_2  : 0.000112535091441  6.21209090165e-06  
CD2_1 CD2_2  : -6.21049551301e-06  0.000112564000117  
NAXIS : 4096  4096
results:

 id      xcenter       ...    aperture_sum     aperture_sum_err 
           pix         ...                                      
--- ------------------ ... ------------------ ------------------
  1  2825.058367430467 ...   2836635.92219475 1836.9813277095598
  2 2070.5088428664767 ...  33560.11886466219  738.7273446109216
  3  2333.906269962376 ...  49801.29145707819  739.9480923051497
  4  2168.696896001486 ...  30623.07402119023  721.1066852266567
  5  417.0714197003731 ... 178924.48469137884  825.5575915140222
  6  1060.293464644735 ... 417143.25518325117  956.2245851457928
  7 1337.3099229917461 ...  44155.86970530165   734.281508494764
  8 1993.9127498770063 ...  295542.6363242051  893.9424791328746
  9  3106.370

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
